In [1]:


from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from datasets import DatasetDict
import datasets

In [3]:
#Here is where the fine tuning data gets loaded
dataset_train = load_dataset('csv',data_files='bin18_25k.csv')
dataset_train=dataset_train['train']
#dataset_train=dataset_train['train'].rename_column("Sequence", "Seq")
#dataset_train=dataset_train.rename_column("Exp", "Actual_Exp")






Using custom data configuration default-2e70721c8fb6e7d8
Reusing dataset csv (/nfs/home/zlu21/.cache/huggingface/datasets/csv/default-2e70721c8fb6e7d8/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
#Train test split
dataset=dataset_train.train_test_split(test_size=00.1, shuffle=True)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Actual Expression', 'k-mer_occurs/size', 'median k-mer occurance', 'max k-mer occurance', 'Unnamed: 0', 'Seq', 'Pred', 'Exp', 'Residual'],
        num_rows: 309605
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Actual Expression', 'k-mer_occurs/size', 'median k-mer occurance', 'max k-mer occurance', 'Unnamed: 0', 'Seq', 'Pred', 'Exp', 'Residual'],
        num_rows: 34401
    })
})

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Actual Expression', 'k-mer_occurs/size', 'median k-mer occurance', 'max k-mer occurance', 'Unnamed: 0', 'Seq', 'Pred', 'Exp', 'Residual'],
        num_rows: 309605
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Actual Expression', 'k-mer_occurs/size', 'median k-mer occurance', 'max k-mer occurance', 'Unnamed: 0', 'Seq', 'Pred', 'Exp', 'Residual'],
        num_rows: 34401
    })
})

Below we specify the model and tokenizer, you can swap 

In [7]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader


BASE_MODEL = "bert-base-uncased"
LEARNING_RATE = 1e-5
MAX_LENGTH = 128
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

trainedModelPath="./bert_epoch3_r52"
model = AutoModelForSequenceClassification.from_pretrained(trainedModelPath,num_labels=1)

In [8]:
#Defining the function that splits the sequences into nuclitides seperated by spaces
from re import findall
import itertools  
def textProcess(s,k,fill=' '):
  return ' '.join(map(''.join, itertools.zip_longest(*[iter(s)]*k, fillvalue=fill)))

In [9]:
#Preprocess the input and tokenize the sequences, note that only the sequences and their expression values are used

def preprocess_function(examples):
    text=examples['Seq']
    label=examples['Exp']
    text=textProcess(text,1)
    examples["text"]=text
    examples = tokenizer(text, truncation=True, padding="max_length", max_length=MAX_LENGTH)
    # Change this to real number
    examples["label"] = float(label)
    return examples

tokenized_dataset=DatasetDict()
for split in dataset:
    tokenized_dataset[split] = dataset[split].map(preprocess_function,num_proc=48)

#0:   0%|          | 0/6451 [00:00<?, ?ex/s]

#1:   0%|          | 0/6451 [00:00<?, ?ex/s]

#2:   0%|          | 0/6451 [00:00<?, ?ex/s]

#3:   0%|          | 0/6451 [00:00<?, ?ex/s]

#4:   0%|          | 0/6451 [00:00<?, ?ex/s]

...

#43:   0%|          | 0/716 [00:00<?, ?ex/s]

#44:   0%|          | 0/716 [00:00<?, ?ex/s]

#45:   0%|          | 0/716 [00:00<?, ?ex/s]

#46:   0%|          | 0/716 [00:00<?, ?ex/s]

#47:   0%|          | 0/716 [00:00<?, ?ex/s]

In [ ]:
def selectColumns(dataset,columns=["input_ids","token_type_ids","attention_mask","label"]):
    cols_to_remove= dataset.column_names
    for c in columns:
        cols_to_remove.remove(c)
    return dataset.remove_columns(cols_to_remove)





In [ ]:
for split in tokenized_dataset:
    tokenized_dataset[split]=selectColumns(tokenized_dataset[split])



In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 309605
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 34401
    })
})

In [13]:
!nvidia-smi -L


GPU 0: NVIDIA A40 (UUID: GPU-ff27e5c0-dfa7-5051-ab99-8cc865ee9aae)
GPU 1: NVIDIA A40 (UUID: GPU-5688da18-d5ae-9454-2fff-ded0abfb29e7)
GPU 2: NVIDIA A40 (UUID: GPU-0479f7ae-005d-dc81-4d0d-c19f38133fb8)
GPU 3: NVIDIA A40 (UUID: GPU-051d093e-64e2-d9ed-ac6a-4301da29526c)


In [14]:

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    return {"mse": mse, "mae": mae, "r2": r2}


Here are the trainning arguments:

In [15]:
from transformers import TrainingArguments

EPOCHS = 3
BATCH_SIZE=32
training_args = TrainingArguments(
    f"zluvolyote/DEREXP",
    save_total_limit=3,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    optim="adamw_torch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="r2",
    load_best_model_at_end=True,
    weight_decay=0.01,
    dataloader_num_workers= 2,
    remove_unused_columns=True,

)

Make sure your cuda version supports 'sm_86'

In [16]:
import torch
print(torch.cuda.get_arch_list())

['sm_37', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86']


In [17]:
import torch  

#A simple regression trainer with huber loss
from transformers import Trainer
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.huber_loss(logits, labels.float())
        return (loss, outputs) if return_outputs else loss

In [18]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics_for_regression,
    tokenizer=tokenizer,
)



In [19]:
!nvidia-smi

Sun Aug  7 22:15:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:17:00.0 Off |                    0 |
|  0%   45C    P0    84W / 300W |   2937MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A40          On   | 00000000:65:00.0 Off |                    0 |
|  0%   

In [20]:
device = torch.device("cuda")
print(torch.cuda.current_device())
print(torch.cuda.device_count())

0
2


In [23]:

trainer.train()

***** Running training *****
  Num examples = 309605
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 14514
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Mse,Mae,R2
1,1.219600,1.259101,5.653554,1.681968,0.674769
2,1.207500,1.248608,5.522295,1.672529,0.682320
3,1.191400,1.251767,5.543555,1.675716,0.681096


***** Running Evaluation *****
  Num examples = 34401
  Batch size = 64
Saving model checkpoint to zluvolyote/DEREXP/checkpoint-4838
Configuration saved in zluvolyote/DEREXP/checkpoint-4838/config.json
Model weights saved in zluvolyote/DEREXP/checkpoint-4838/pytorch_model.bin
tokenizer config file saved in zluvolyote/DEREXP/checkpoint-4838/tokenizer_config.json
Special tokens file saved in zluvolyote/DEREXP/checkpoint-4838/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 34401
  Batch size = 64
Saving model checkpoint to zluvolyote/DEREXP/checkpoint-9676
Configuration saved in zluvolyote/DEREXP/checkpoint-9676/config.json
Model weights saved in zluvolyote/DEREXP/checkpoint-9676/pytorch_model.bin
tokenizer config file saved in zluvolyote/DEREXP/checkpoint-9676/tokenizer_config.json
Special tokens file saved in zluvolyote/DEREXP/checkpoint-9676/special_tokens_map.json
Deleting older checkpoint [zluvolyote/DEREXP/checkpoint-1500] due to args.save_total_limit
***** R

TrainOutput(global_step=14514, training_loss=1.21022108045675, metrics={'train_runtime': 3221.5099, 'train_samples_per_second': 288.317, 'train_steps_per_second': 4.505, 'total_flos': 6.109482517034112e+16, 'train_loss': 1.21022108045675, 'epoch': 3.0})

In [26]:
trainer.save_model("./final_fine_tuned")

Saving model checkpoint to ./final_fine_tuned
Configuration saved in ./final_fine_tuned/config.json
Model weights saved in ./final_fine_tuned/pytorch_model.bin
tokenizer config file saved in ./final_fine_tuned/tokenizer_config.json
Special tokens file saved in ./final_fine_tuned/special_tokens_map.json


Code below is used for generating the predictions for provided test set:

In [27]:
dataset_test = load_dataset('zluvolyote/DreamNLP',split='test')

Using custom data configuration zluvolyote--DreamNLP-c8737e06b2c1a369
Reusing dataset text (/nfs/home/zlu21/.cache/huggingface/datasets/zluvolyote___text/zluvolyote--DreamNLP-c8737e06b2c1a369/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


In [28]:
dataset_test

Dataset({
    features: ['text'],
    num_rows: 71103
})

In [29]:
def predictFromText(text):
  encoded = tokenizer(text, truncation=True, padding="max_length", max_length=128, return_tensors="pt").to('cuda')
  logits = model(**encoded).logits
  return logits.cpu().detach().numpy().flatten()

In [ ]:
textList=[]
predicted=[]
labels=[]

testSet = dataset_test
cnt=0
for entry in testSet:
    text=entry['text']
    textList.append(text)
    text=textProcess(text,1)
    pred=predictFromText(text)
    predicted.append(pred[0])
    cnt+=1
    #a really simple progress bar
    if cnt%1000==0:
        print(cnt)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000


In [ ]:
import pandas as pd

df=pd.DataFrame()
df['Seq']=textList
df['Pred']=predicted



In [ ]:
df.to_csv('final_submission.csv',header=False,index=False)